# Project Report

In [1]:
# Run to load packages
library(tidyverse)
library(repr)
library(dplyr)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks read

# Introduction:

Tennis is a popular sport among men and women all over the world. Therefore, the analysis of factors affecting performance and success is of great interest. Since tennis players' earnings can vary widely and "Prize Money” could reflect a player's earnings in tennis tournaments, understanding the determinants of prize money can provide insights into the financial success of professional tennis players. One main factor is ranking points. In professional tennis, players compete in various tournaments, accumulate ranking points, and move up or down in the rankings based on their performance. This system is used to determine tournament entries and player recognition. In our model, our research question is "How can we use ranking points to predict the prize money a tennis player receives?" To answer the question, we will be using the data `player_stats(1).csv` from Ultimate Tennis Statistics. This data set contains the statistics of the top 500 tennis players in the world.



****Working on more*****

Cited Source: https://olympics.com/en/news/tennis-rankings-atp-wta-men-women-doubles-singles-system-grand-slam-olympics

# Data Description:

We use data from Ultimate Tennis Statistics, which is based on the open-source tennis data repository by Jeff Sackmann with some corrections and additions for incorrect or missing data. This database provides users with a variety of tennis statistics for men's tennis during the Open period, such as historical results, in-match statistics, records or all-time greats comparisons. The extensive nature of the dataset makes it a valuable resource for conducting in-depth analyses and deriving insights into various facets of men's tennis during the specified Open period.

The original dataset contains 500 rows and 38 columns with 500 tennis players and 37 factors of interest. We have wrangled and cleaned the dataset so that we can focus only on the variables we want to explore (age, backhand use, best rank, current rank, dominant hand, and prize money). We have also removed all the "NA" values. The wrangled dataset contains 407 rows and 9 columns. 

Our dependent variable is a player's earnings in tennis tournaments which is represented by `prize_money` in our model (Unit: US dollar). Since prize money is a visual reflection of tennis players' income in the game, we set prize money as the independent variable. We predict the prize money income of professional tennis players in the subsequent data analysis to understand their economic success.

Our main independent variable are `best_rank` and `current_rank`; `best_rank` represents a player's the highest rank they have achieved. It is crucial to consider this variable as it can impact a player's reputation, endorsements, and invitations to play in future prestige tournaments. The `best_rank` variable also provides insight into a player's playing history, which shows the challenges they faces or if they are on an upward trajectory. `current_rank` represents a tennis player's current rank. By analyzing the current ranks of players, it allows us to analyze how their ranking point may affect their progress; having a higher current rank will allow tennis players an opportunity to enter more honored tournaments, which will impact the amount of prize money they will earn. The independent variables are both crucial to analyze our problem because of a player has a higher rank, the amount of prize money they will receive may vary. 

Our control variables are `age`, `dominant_dummy` (their playing hand), and `backhand_dummy` (the use of backhand plays). These variables were chosen as they may all lead to changes of our dependent variable,`prize_money`.
`age` is important since younger players may have more energy and stamina, potentially providing them with a competitive advantage. On the other hand, older players may have the experience and strategic depth, but their physical condition may decline. As for `dominant_dummy` and `backhand_dummy`, these variables represent if they play plays with their left or right hand. Dominant hand and the choice of backhand style can impact a player's style and game strategy, which might be reflected in their prize earnings. We transformed the qualitative variables to its respective dummy variables, 1 is for right handed while 0 is for left handed. All of these variables provide different dimensions of a player's career and can help create a more accurate model.

We have conducted a summary statistics table. For the "Variable" column, it lists the names or labels of the variables or features in our dataset. 
For the "Mean" column, it displays the average value of each variable. The mean is a measure of central tendency, representing the central or typical value in our dataset.
For the "Max" column, it shows the maximum value observed for each variable. This value represents the highest point or measurement in the dataset for a particular variable. It helps identify the upper limit or extreme values within the dataset.
For the "Min" column, it shows the minimum value observed for each variable. This value represents the lowest point or measurement in the dataset for a particular variable. It helps identify the lowest limit or extreme values within the dataset.


`age` - Player's age

`backhand_dummy` - Player's backhand style	(where 1 means "One-handed" and 0 means "Two-handed")

`best_rank` - Player's highest rank

`current_rank` - Player's current standing

`dominant_dummy` - Player's dominant hand (1 means "Right-handed" and 0 means "Left-handed)

`prize_money` - Player's prize money

Cited Source: https://www.ultimatetennisstatistics.com/about

In [2]:
# run to load data set
player_data <- read_csv("https://raw.githubusercontent.com/dcai01/dsci100_project/main/project_proposal/data/player_stats%20(1).csv")
num_rows <- nrow(player_data)
num_rows
num_cols <- ncol(player_data)
num_cols

New names:
• `` -> `...1`
Rows: 500 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (25): Age, Country, Plays, Wikipedia, Current Rank, Best Rank, Name, Bac...
dbl (13): ...1, Turned Pro, Seasons, Titles, Best Season, Retired, Masters, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 500

[1] 38

In [3]:
player_data

#The original dataset contains 500 rows and 38 columns with 500 tennis players and 37 factors of interest.

...1,Age,Country,Plays,Wikipedia,Current Rank,Best Rank,Name,Backhand,Prize Money,⋯,Facebook,Twitter,Nicknames,Grand Slams,Davis Cups,Web Site,Team Cups,Olympics,Weeks at No. 1,Tour Finals
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
0,26 (25-04-1993),Brazil,Right-handed,Wikipedia,378 (97),363 (04-11-2019),Oscar Jose Gutierrez,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,18 (22-12-2001),United Kingdom,Left-handed,Wikipedia,326 (119),316 (14-10-2019),Jack Draper,Two-handed,"$59,040",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,32 (03-11-1987),Slovakia,Right-handed,Wikipedia,178 (280),44 (14-01-2013),Lukas Lacko,Two-handed,"US$3,261,567",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,21 (29-05-1998),"Korea, Republic of",Right-handed,Wikipedia,236 (199),130 (10-04-2017),Duck Hee Lee,Two-handed,"$374,093",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,27 (21-10-1992),Australia,Right-handed,Wikipedia,183 (273),17 (11-01-2016),Bernard Tomic,Two-handed,"US$6,091,971",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,22 (11-02-1997),Poland,Right-handed,Wikipedia,31 (1398),31 (20-01-2020),Hubert Hurkacz,Two-handed,"$1,517,157",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,28 (18-11-1991),United States,Right-handed,Wikipedia,307 (131),213 (31-10-2016),Sekou Bangoura,Two-handed,"$278,709",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
7,21 (12-05-1998),"Taiwan, Province of China",Right-handed,Wikipedia,232 (205),229 (04-11-2019),Tung Lin Wu,Two-handed,"$59,123",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
8,25 (29-07-1994),Uzbekistan,Right-handed,Wikipedia,417 (81),253 (17-07-2017),Sanjar Fayziev,Two-handed,"$122,734",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


Run the next cell tidy up the data.

In [4]:
set.seed(1000)
# Run to view tidied/wrangled data
player_data$"Prize Money" <- gsub("[^0-9.-]", "", player_data$"Prize Money")

player_data_renamed <- player_data |>
    separate(Age, into = c("age", "birthday"), sep ="\\(") |>
    separate("Best Rank", into = c("best_rank", "date"), sep ="\\(") |>
    separate("Current Rank", into = c("current_rank", "points"), sep ="\\(") |>
    separate("Prize Money", into = c("prize_money", "dash"), sep = "-")|>
    rename(plays = Plays, backhand = Backhand, name = Name)|>
    select(name, age, plays, current_rank, best_rank, backhand, prize_money) |>
    drop_na()

player_data_renamed$dominant_dummy <- ifelse(player_data_renamed$plays == "Right-handed", 1, 0) #1 means "Right-handed" and 0 means "Left-handed	
player_data_renamed$backhand_dummy <- ifelse(player_data_renamed$backhand == "One-handed", 1, 0) #1 means "One-handed	" and 0 means "Two-handed"

player_data_renamed <- player_data_renamed |>
                       mutate(age = as.numeric(age),
                              current_rank = as.numeric(current_rank),
                              best_rank = as.numeric(best_rank),
                              prize_money = as.numeric(prize_money),
                              plays = as.numeric(plays),
                              backhand = as.numeric(backhand))

player_data_renamed

Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 399 rows [2, 3, 4, 5, 6,
7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21, 22, 23, ...].”
Warning message:
“There were 2 warnings in `mutate()`.
The first warning was:
ℹ In argument: `plays = as.numeric(plays)`.
Caused by warning:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 1 remaining warning.”


name,age,plays,current_rank,best_rank,backhand,prize_money,dominant_dummy,backhand_dummy
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Jack Draper,18,NA,326,316,NA,59040,0,0
Lukas Lacko,32,NA,178,44,NA,3261567,1,0
Duck Hee Lee,21,NA,236,130,NA,374093,1,0
Bernard Tomic,27,NA,183,17,NA,6091971,1,0
Hubert Hurkacz,22,NA,31,31,NA,1517157,1,0
Sekou Bangoura,28,NA,307,213,NA,278709,1,0
Tung Lin Wu,21,NA,232,229,NA,59123,1,0
Sanjar Fayziev,25,NA,417,253,NA,122734,1,0
Emil Ruusuvuori,20,NA,104,104,NA,74927,1,0


Run the cell below to select the columnms we are interested in using for our analysis into `player_data_selected`.

In [5]:
player_data_selected<-as_tibble(player_data_renamed)|>
                    select(age, current_rank, best_rank, prize_money, dominant_dummy, backhand_dummy, plays, backhand) 
head(player_data_selected)

age,current_rank,best_rank,prize_money,dominant_dummy,backhand_dummy,plays,backhand
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
18,326,316,59040,0,0,NA,NA
32,178,44,3261567,1,0,NA,NA
21,236,130,374093,1,0,NA,NA
27,183,17,6091971,1,0,NA,NA
22,31,31,1517157,1,0,NA,NA
28,307,213,278709,1,0,NA,NA


Before we perform our exploratory data analysis, we are going to create our training and testing data sets. Our training data will be called `player_train` and our testing data will be called `player_test`.

In [6]:
# Run to split data into training and testing set
player_split <- initial_split(player_data_selected, prop = 0.75, strata = prize_money)
player_train <- training(player_split)
player_test <- testing(player_split)

player_train

age,current_rank,best_rank,prize_money,dominant_dummy,backhand_dummy,plays,backhand
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
18,326,316,59040,0,0,NA,NA
25,417,253,122734,1,0,NA,NA
22,339,312,87942,1,0,NA,NA
26,448,293,32892,1,0,NA,NA
24,297,291,100014,1,0,NA,NA
21,228,228,43346,1,0,NA,NA
20,331,187,127760,0,0,NA,NA
22,189,173,39679,1,0,NA,NA
21,320,233,49635,1,0,NA,NA


Next, we are summarizing the player data into its mean values (`player_summarize_mean`), maximum values (`player_summarize_max`), and minimum values (`player_summarize_min`). After performing the summarization of our player data, we are merging the data together by Variables (i.e. backhand, age, rank, etc.) into `player_summarize_data`.

In [11]:
# Run to calculate mean, max and min of each variable
player_summarize_mean <- player_train |>
                         select (-plays)|>
                         select(-backhand)|>
                        map_dfr(mean, na.rm = TRUE) |>
                        round(1) |>
                         pivot_longer(
                          cols = age:backhand_dummy,
                          names_to = "Variable",
                          values_to = "Mean")

                        
player_summarize_max <- player_train |>
                        select (-plays)|>
                         select(-backhand)|>
                        map_dfr(max, na.rm = TRUE)|>
                        pivot_longer(
                        cols = age:backhand_dummy,
                        names_to = "Variable",
                        values_to = "Max")

player_summarize_min <- player_train |>
                       select (-plays)|>
                         select(-backhand)|>
                        map_dfr(min, na.rm = TRUE)|>
                        pivot_longer(
                        cols = age:backhand_dummy,
                        names_to = "Variable",
                        values_to = "Min") 

player_summarize_data_v1 = merge(player_summarize_mean, player_summarize_max, by = "Variable")
player_summarize_data_v2 = merge(player_summarize_data_v1,player_summarize_min, by = "Variable")


player_summarize_data_v2




Variable,Mean,Max,Min
<chr>,<dbl>,<dbl>,<dbl>
age,26.4,40,17
backhand_dummy,0.1,1,0
best_rank,145.9,488,1
current_rank,223.6,500,1
dominant_dummy,0.9,1,0
prize_money,77568262.8,2746601610,2980


From the summarization of the variables, it allows us to analyze average, maximum, and minimum value of each variable. For `age`, the average age of tennis players is 26 years old, while the oldest player is 40 and the youngest player is 17. The average `best_rank` of players is 146th place, with the lowest rank being 488th place and the highest rank being 1st place. The `current_rank`of players is 224th place, with the lowest rank being 500th place and the highest rank being 1st place. With this analysis, we summarized that the average player prize money being $\$ $77568263, with the largest amount of money received being $\$ $2746601610 and the lowest amount of money received being $\$ $2980.

# Methods & Results:

We will use a regression model to predict the expected prize money for a tennis athlete by using multiple variables to answer our research question.


We use scatter plots and box plots to visualize the results, which depict the relationships between independent variables with prize money as the dependent variable. 
By examining scatter plots, we can gain valuable insights, such as if the points cluster in a specific pattern, it may suggest a correlation or relationship between independent variables and prize money. Additionally, the color-coded data points help us distinguish players with different dominant hands, potentially revealing variations in prize money earnings based on this characteristic. (*******more work*******)

By examining box plots, we...
Figure 4 shows that 
Figure 5

- describe in written English the methods you used to perform your analysis from beginning to end that narrates the code the does the analysis.
- your report should include code which:
- loads data from the original source on the web 
- wrangles and cleans the data from it's original (downloaded) format to the format necessary for the planned analysis
- performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis 
- creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis
- performs the data analysis
- creates a visualization of the analysis 
    - note: all tables and figure should have a figure/table number and a legend


We firstly use the `ggpairs()` function from the GGally package to explore the relationships between the different variables. 

In [ ]:
install.packages("GGally")
library(GGally)

In [ ]:
set.seed(1000) # DO NOT REMOVE

options(repr.plot.height = 8, repr.plot.width = 9)
 player_pairplot <- player_train |> 
                         select (-plays)|>
                         select(-backhand)|>
     ggpairs(
         lower = list(continuous = wrap('points', alpha = 0.4)),
         diag = list(continuous = "barDiag")
     ) +
     theme(text = element_text(size = 8))

player_pairplot
#Figure 0

From Figure 0, we could explore the relationship between the different variables. `age` has a positive correlation (0.246) with `prize_money`; `current_rank` has a negative correlation (-0.263) with `prize_money`; `best_rank` has a negative correlation (-0.239) with `prize_money`.

In [ ]:
# Run to see plot comparing prize money and current rank

options(repr.plot.width = 10, repr.plot.height = 5) 
small_plot_current <- ggplot(player_train, aes(x = current_rank, y = prize_money)) +
  geom_point() +
  xlab("Current Rank") +
  xlim(0, 500) +
  ylim(0, 2500000) +
  ylab("Prize Money (USD)") +
  theme(text = element_text(size = 12)) +
  ggtitle("Prize Money (USD) VS Current Rank (Figure 1)")

small_plot_current

#Figure 1

In [ ]:
# Run to see plot comparing prize money and best rank 

options(repr.plot.width = 10, repr.plot.height = 5) 
small_plot_best <- ggplot(player_train, aes(x = best_rank, y = prize_money)) +
  geom_point() +
  xlab("Best Rank") +
  xlim(0, 500) +
  ylim(0, 2500000) +
  ylab("Prize Money (USD)") +
  theme(text = element_text(size = 12)) +
  ggtitle("Prize Money (USD) VS Best Rank (Figure 2)")

small_plot_best


In [ ]:
# Run to see plot comparing prize money and age

options(repr.plot.width = 10, repr.plot.height = 5) 
small_plot_age <- ggplot(player_train, aes(x = age, y = prize_money)) +
  geom_point() +
  xlab("Age") +
  xlim(12, 50) +
  ylim(0, 2500000) +
  ylab("Prize Money (USD)") +
  theme(text = element_text(size = 12)) +
  ggtitle("Prize Money (USD) VS Age (Figure 3)")


small_plot_age

In [ ]:
# Run to see plot comparing prize money and dominant hand

options(repr.plot.width = 7, repr.plot.height = 5) 
small_plot_hand <- ggplot(player_train, aes(x = dominant_dummy, y = prize_money, color = plays)) +
  geom_boxplot(alpha=0.4) +
  xlab("Dominant Hand") +
  xlim(0,1) +
  ylim(0, 2500000) +
  scale_x_continuous(breaks = c(0, 1), labels = c("Left-handed", "Right-handed")) +
  ylab("Prize Money (USD)") +
  labs(color = "Dominant Hand") +
  theme(text = element_text(size = 12)) +
  ggtitle("Prize Money (USD) VS Dominant Hand (Figure 4)")

small_plot_hand

The boxplot shows that left-handed players have very similar median (middle quartile) of prize money (about $260000 ~ $280000) compare with right-handed players. The distribution of the variance in prizes is larger for left-handed players than for right-handed players. The top 25% of left-handed player have relatively higher prize money, even though right-handed group tends to have more possibile outliers.

In [ ]:
# Run to see plot comparing prize money and backhand use

options(repr.plot.width = 7, repr.plot.height = 5) 
small_plot_backhand <- ggplot(player_train, aes(x = backhand_dummy, y = prize_money, color = backhand)) +
  geom_boxplot(alpha=0.4) +
  xlab("Backhand") +
  xlim(0,1) +
  ylim(0, 2500000) +
  scale_x_continuous(breaks = c(0, 1), labels = c("One-handed", "Two-handed")) +
  ylab("Prize Money (USD)") +
  labs(color = "Backhand") +
  theme(text = element_text(size = 12)) +
  ggtitle("Prize Money (USD) VS Backhand (Figure 5)")

small_plot_backhand

The boxplot shows that left-handed players have very similar median (middle quartile) of prize money (about $260000 ~ $280000) compare with right-handed players. The distribution of the variance in prizes is larger for left-handed players than for right-handed players. The top 25% of left-handed player have relatively higher prize money, even though right-handed group tends to have more possibile outliers.

## Discussion:
- summarize what you found
- discuss whether this is what you expected to find?
- discuss what impact could such findings have?
- discuss what future questions could this lead to?

## References
- At least 2 citations of literature relevant to the project (format is your choice, just be consistent across the references).
- Make sure to cite the source of your data as well.